# Experimento: DIN - Mondial - Foreign Key - GPT 3.5

In [ ]:

from langchain.chat_models import ChatOpenAI
from urllib.parse import quote  

from dotenv import load_dotenv
import time
import os
import sys
import json
load_dotenv()


from din_schema_linking import schema_linking_module
from din_classification import classification_module
from din_generating_sql_by_type import generating_sql_by_type_module
from din_self_correction import self_correction_module

experiment_path = '..\..'
path = os.path.abspath('')
module_path = os.path.join(path, experiment_path)
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")


from sqldatabase_langchain_utils import SQLDatabaseLangchainUtils

In [2]:
SCHEMA = 'mondial_gpt'
PREFIX = 'mondial'
FILE_NAME_RESULT = f"results/5_din_queries_chatgpt_{SCHEMA}_fk.json"

In [3]:
def save_queries(queries):
    data = {"queries":queries}
    with open(FILE_NAME_RESULT, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 
        
def read_queries():
    with open(FILE_NAME_RESULT, encoding='utf-8', errors='ignore') as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries


## Conexão com o banco

In [ ]:
json_file_path = f"{experiment_path}/datasets/{SCHEMA}_db_connection.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    db_connection = json.load(json_data, strict=False)
db_connection

In [5]:
db = SQLDatabaseLangchainUtils(db_connection=db_connection)

exclusao = [
    f"{SCHEMA}_tmdp",
    f"{SCHEMA}_tmdpmap",
    f"{SCHEMA}_tmds",
    f"{SCHEMA}_tmjmap",
    f"{SCHEMA}_tpv",
    f"{SCHEMA}_tmdc",
    f"{SCHEMA}_tmdcmap",
    f"{SCHEMA}_tmdej",
    f"{SCHEMA}_log_action",
    f"{SCHEMA}_log_error",
    f"{SCHEMA}_favorite_item", 
    f"{SCHEMA}_favorite_query",
    f"{SCHEMA}_favorite_tag",
    f"{SCHEMA}_favorite_tag_item",
    f"{SCHEMA}_favorite_visualization",
    f"{SCHEMA}_dashboard",
    f"{SCHEMA}_history",
    "teste_cliente",
    "teste_fornecedor",
    "teste_funcionario"
]

include_tables = [s for s in db.get_table_names() if not s.startswith(PREFIX) and s not in exclusao]
db = SQLDatabaseLangchainUtils(db_connection=db_connection, include_tables=include_tables)
db.get_table_names()

c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:112: SAWarning: Did not recognize type 'GEOCOORD' of column 'coordinates'
  self._metadata.reflect(
c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:112: SAWarning: Did not recognize type 'GEOCOORD' of column 'source'
  self._metadata.reflect(
c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:112: SAWarning: Did not recognize type 'GEOCOORD' of column 'estuary'
  self._metadata.reflect(


['airport',
 'borders',
 'city',
 'citylocalname',
 'cityothername',
 'citypops',
 'continent',
 'country',
 'countrylocalname',
 'countryothername',
 'countrypops',
 'desert',
 'economy',
 'encompasses',
 'ethnicgroup',
 'geo_desert',
 'geo_estuary',
 'geo_island',
 'geo_lake',
 'geo_mountain',
 'geo_river',
 'geo_sea',
 'geo_source',
 'island',
 'islandin',
 'ismember',
 'lake',
 'lakeonisland',
 'language',
 'located',
 'locatedon',
 'mergeswith',
 'mountain',
 'mountainonisland',
 'organization',
 'politics',
 'population',
 'province',
 'provincelocalname',
 'provinceothername',
 'provpops',
 'religion',
 'river',
 'riveronisland',
 'riverthrough',
 'sea']

## DIN - Function

In [6]:
def run_din(question, db, model_name = 'gpt-3.5-turbo-16k', debbug = False, callback=None):
    
    model_kwargs = {'top_p':1.0, 'frequency_penalty':0.0, 'presence_penalty':0.0, 'stop':['Q:']}
    llm = ChatOpenAI(model_name = model_name, temperature=0.0, n=1, max_tokens=600, model_kwargs = model_kwargs)

    model_kwargs['stop'] = ['#', ';','\n\n']
    llm_fix = ChatOpenAI(model_name = model_name, temperature=0.0, n=1, max_tokens=350, model_kwargs = model_kwargs)
    
    schema_links = schema_linking_module(db, llm, question, callback=callback)
   
    classification = classification_module(db, llm, question, schema_links, callback=callback)
    
    SQL = generating_sql_by_type_module(db, llm, question, schema_links, classification, callback=callback)
   
    SQL_FINAL = self_correction_module(db, llm_fix, question, SQL, callback=callback)
    
    if debbug == True:
        print("Schema Links =", schema_links)
        print('Class: ', classification['predicted_class'])
        print('SQL >', SQL)
        print('SQL Final >', SQL_FINAL)
        print()
        print('-------')
    return SQL_FINAL

## Preparando as consultas 

In [7]:

json_file_path = f"../../datasets/{PREFIX}/queries_{PREFIX}.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    queries = json.load(json_data, strict=False)
queries = queries['queries']
queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': '',
  'type': 'simple'},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': '',
  'type': 'simple'},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': '',
  'type': 'medium'},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': '',
  'type': 'simple'},
 {'id': '5',
  'question': 'What is the total of provinces of Netherlands?',
  'query_string': '',
  'type': 'complex'},
 {'id': '6',
  'question': 'What is the percentage of religious people are hindu in thailand?',
  'query_string': '',
  'type': 'complex'},
 {'id': '7',
  'question': 'List the number of provinces each river flows through.',
  'query_string': '',
  'type': 'medium'},
 {'id': '8',
  'question': 'Find all countries that became independent between 8/1/1910 and 8/1/1950.',
  'query_string': '',
  'type': 'complex'},
 {'id': '9',
  'que

## Tracking token usage

In [8]:
track_token = [] 
def tracking_token(cb =None, reset = False):
    global track_token
    track_token.append(cb)
    if reset:
        track_token = []

In [9]:

def convert_to_dict_tracking_token():
    token_usage = {}
    for e in track_token:
        for key in e.keys():
            token_usage[key] = {}
            token_usage[key]['total_tokens'] = e[key].total_tokens
            token_usage[key]['total_cost'] = e[key].total_cost
            token_usage[key]['prompt_tokens'] = e[key].prompt_tokens
            token_usage[key]['completion_tokens'] = e[key].completion_tokens 
    return token_usage

## Executando o método

In [ ]:
tracking_token(reset=True)
errors = []
query_index = -1
for instance in queries:
    query_index +=1
    try:
        
        start_time = time.time()
        sql = run_din(instance["question"], db, debbug=True, callback=tracking_token)
        end_time = time.time()
        instance["query_string"] = sql
        instance["token_usage"] = convert_to_dict_tracking_token()
        instance['time'] = end_time - start_time
        print(instance['id'], instance['question'], instance["query_string"], instance['time'])
        save_queries(queries)
    except:
        print("Error")
        errors.append(query_index)
        pass
    finally:
        tracking_token(reset=True)



#### Fixing query

In [ ]:
pos = 40
instance = queries[pos]
q = read_queries()
start_time = time.time()
sql = run_din(instance["question"], db, callback=tracking_token)
end_time = time.time()
instance["query_string"] = sql
instance["token_usage"] = convert_to_dict_tracking_token()
instance['time'] = end_time - start_time
q[pos] = instance
save_queries(q)
print(instance['id'], instance['question'], instance["query_string"], instance['time'])



In [11]:
to_fix = [59,62,72,85,99]
for pos in to_fix:
    instance = queries[pos]
    q = read_queries()
    start_time = time.time()
    sql = run_din(instance["question"], db, callback=tracking_token)
    end_time = time.time()
    instance["query_string"] = sql
    instance["token_usage"] = convert_to_dict_tracking_token()
    instance['time'] = end_time - start_time
    q[pos] = instance
    save_queries(q)
    print(instance['id'], instance['question'], instance["query_string"], instance['time'])

-- Classification -- 
generations=[[ChatGeneration(text='The SQL query for the question "List the names of capital cities which are the base for organizations in alphabetical order" needs these tables = [city, organization], so we need JOIN.\nPlus, it doesn\'t require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""].\nSo, we need JOIN and don\'t need nested queries, then the SQL query can be classified as "NON-NESTED".\nLabel: "NON-NESTED"', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='The SQL query for the question "List the names of capital cities which are the base for organizations in alphabetical order" needs these tables = [city, organization], so we need JOIN.\nPlus, it doesn\'t require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""].\nSo, we need JOIN and don\'t need nested queries, then the SQL query can be classified as "NON-NESTED".\nL

In [ ]:
errors

[]

In [ ]:
queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT country.area FROM country WHERE country.name = 'Thailand'",
  'type': 'simple',
  'token_usage': {'schema_linking': {'total_tokens': 5953,
    'total_cost': 0.017937,
    'prompt_tokens': 5875,
    'completion_tokens': 78},
   'classification': {'total_tokens': 3673,
    'total_cost': 0.011116,
    'prompt_tokens': 3576,
    'completion_tokens': 97},
   'sql_generation': {'total_tokens': 1788,
    'total_cost': 0.005377,
    'prompt_tokens': 1775,
    'completion_tokens': 13},
   'self_correction': {'total_tokens': 2195,
    'total_cost': 0.006600999999999999,
    'prompt_tokens': 2179,
    'completion_tokens': 16}}},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT province.name FROM province WHERE province.area > 10000',
  'type': 'simple',
  'token_usage': {'schema_linking': {'total_tokens': 5988,
    'total_cost': 0.018071000000000004,

In [ ]:
sql = run_din("How deep is Lake Kariba?", db, callback=tracking_token)
print(convert_to_dict_tracking_token())

## Testando o DIN para uma consulta  no Mondial com FK

In [ ]:
question = 'What is the area of Thailand?'

### LLM

In [ ]:
model_name = 'gpt-3.5-turbo-16k'
# model_name = 'gpt-4'

model_kwargs = {'top_p':1.0, 'frequency_penalty':0.0, 'presence_penalty':0.0, 'stop':['Q:']}
llm = ChatOpenAI(model_name = model_name, temperature=0.0, n=1, max_tokens=600, model_kwargs = model_kwargs)

model_kwargs['stop'] = ['#', ';','\n\n']
llm_fix = ChatOpenAI(model_name = model_name, temperature=0.0, n=1, max_tokens=350, model_kwargs = model_kwargs)



### Schema Linking

In [ ]:
schema_links = schema_linking_module(db, llm, question)
print("Schema Links =", schema_links)

### Classification

In [ ]:
classification = classification_module(db, llm, question, schema_links)
print(classification)

{'predicted_class': '"EASY"', 'classification_label': 'The SQL query for the question "What is the area of Thailand?" needs the table [mondial_country], so we don\'t need JOIN.\nPlus, it doesn\'t require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""].\nSo, we don\'t need JOIN and don\'t need nested queries, then the SQL query can be classified as "EASY".\nLabel: "EASY"'}


### Generate SQL

In [ ]:
SQL = generating_sql_by_type_module(db, llm, question, schema_links, classification)
print(SQL)


SELECT area FROM mondial_country WHERE name = 'Thailand'
SELECT area FROM mondial_country WHERE name = 'Thailand'


### Self-Correction

In [ ]:
SQL_FINAL = self_correction_module(db, llm, question, SQL)
print(SQL_FINAL)

SELECT area FROM mondial_country WHERE name = 'Thailand'
SELECT area FROM mondial_country WHERE name = 'Thailand'


In [ ]:
# 3,46 - 3,58
# Uma consulta classificada como easy = $0,12

### Testing Self-Correction 

In [ ]:
SQL_ERROR = "SELECT area  FROM  mondial_country  WHERE  country = 'Thailand'"
print(SQL_ERROR)
print()
SQL_FINAL = self_correction_module(db, llm, question, SQL_ERROR)
print(SQL_FINAL)

SELECT area  FROM  mondial_country  WHERE  country = 'Thailand'

SELECT area FROM mondial_country WHERE name = 'Thailand'
SELECT area FROM mondial_country WHERE name = 'Thailand'


In [ ]:
question = 'Show me all employees for each client'

### Schema Linking

In [ ]:
schema_links = schema_linking_module(db, llm, question)
print("Schema Links =", schema_links)

Schema Links = [teste_funcionario.*, teste_cliente.*, teste_cliente.idfuncionarioresponsavel=teste_funcionario.idfuncionario]


### Classification 

In [ ]:
classification = classification_module(db, llm, question, schema_links)
print(classification)

{'predicted_class': '"NON-NESTED"', 'classification_label': 'The SQL query for the question "Show me all employees for each client" needs these tables = [teste_funcionario, teste_cliente], so we need JOIN.\nPlus, it doesn\'t require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""].\nSo, we need JOIN and don\'t need nested queries, then the the SQL query can be classified as "NON-NESTED".\nLabel: "NON-NESTED"'}
{'predicted_class': '"NON-NESTED"', 'classification_label': 'The SQL query for the question "Show me all employees for each client" needs these tables = [teste_funcionario, teste_cliente], so we need JOIN.\nPlus, it doesn\'t require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""].\nSo, we need JOIN and don\'t need nested queries, then the the SQL query can be classified as "NON-NESTED".\nLabel: "NON-NESTED"'}


### SQL

In [ ]:
SQL = generating_sql_by_type_module(db, llm, question, schema_links, classification)
print(SQL)

NON-NESTED
SELECT T1.*, T2.* FROM teste_funcionario T1 JOIN teste_cliente T2 ON T1.idfuncionario = T2.idfuncionarioresponsavel
SELECT T1.*, T2.* FROM teste_funcionario T1 JOIN teste_cliente T2 ON T1.idfuncionario = T2.idfuncionarioresponsavel


### Self-Correction

In [ ]:
SQL_FINAL = self_correction_module(db, llm, question, SQL)
print(SQL_FINAL)

SELECT T1.nome AS employee_name, T2.nome AS client_name FROM teste_funcionario T1 JOIN teste_cliente T2 ON T1.idfuncionario = T2.idfuncionarioresponsavel
SELECT T1.nome AS employee_name, T2.nome AS client_name FROM teste_funcionario T1 JOIN teste_cliente T2 ON T1.idfuncionario = T2.idfuncionarioresponsavel


In [ ]:
db.db.run(SQL_FINAL)

"[('João Silva', 'Empresa A'), ('João Silva', 'Cliente C'), ('Maria Santos', 'Cliente B')]"